In [1]:
from osgeo import gdal
import os
import glob
import math
import time

In [2]:
#获取影像的左上角和右下角坐标
def GetExtent(in_fn):
    ds=gdal.Open(in_fn)
    geotrans=list(ds.GetGeoTransform())
    xsize=ds.RasterXSize 
    ysize=ds.RasterYSize
    min_x=geotrans[0]
    max_y=geotrans[3]
    max_x=geotrans[0]+xsize*geotrans[1]
    min_y=geotrans[3]+ysize*geotrans[5]
    ds=None
    return min_x,max_y,max_x,min_y

In [3]:
def GetImageInform(in_files):
    in_fn=in_files[0]
    #获取待镶嵌栅格的最大最小的坐标值
    min_x,max_y,max_x,min_y=GetExtent(in_fn)
    for in_fn in in_files[1:]:
        minx,maxy,maxx,miny=GetExtent(in_fn)
        min_x=min(min_x,minx)
        min_y=min(min_y,miny)
        max_x=max(max_x,maxx)
        max_y=max(max_y,maxy)

    #计算镶嵌后影像的行列号
    in_ds=gdal.Open(in_fn)
    geotrans=list(in_ds.GetGeoTransform())

    geoproj=in_ds.GetProjection()
    width=geotrans[1]
    height=geotrans[5]
    columns=math.ceil((max_x-min_x)/width)
    rows=math.ceil((max_y-min_y)/(-height))
    in_band=in_ds.RasterCount
    geotrans[0]=min_x
    geotrans[3]=max_y

    return columns,rows,in_band,geotrans,geoproj

In [4]:
def MosaicImage(InputPath,OutputPathName):
    os.chdir(InputPath)
#     if os.path.exists('LaiParamOneCycle_2017.tif'):
#         os.remove('LaiParamOneCycle_2017.tif')
    in_files=glob.glob("*.tif")

    columns,rows,in_band,geotrans,geoproj=GetImageInform(in_files)
    out_ds=gdal.GetDriverByName('GTiff').Create(OutputPathName,columns,rows,in_band,gdal.GDT_Float32)
    out_ds.SetProjection(geoproj)
    out_ds.SetGeoTransform(geotrans)
    #定义仿射逆变换
    inv_geotrans=gdal.InvGeoTransform(geotrans)
    #开始逐渐写入
    for in_fn in in_files:
        in_ds=gdal.Open(in_fn)
        in_gt=in_ds.GetGeoTransform()
        #仿射逆变换
        offset=gdal.ApplyGeoTransform(inv_geotrans,in_gt[0],in_gt[3])
        x,y=map(int,offset)
        for i in range(in_band):
            if(i==21):
                name='x1'
            elif(i==22):
                name='x2'    
            else:
                name='param'+str(math.floor(i/7)+1)+'_'+'p'+str(i % 7)
            
            data=in_ds.GetRasterBand(i+1).ReadAsArray()
            out_ds.GetRasterBand(i+1).WriteArray(data,x,y)#x，y是开始写入时左上角像元行列号
            # out_ds.GetRasterBand(i+1).SetDescription("%s"%name)
            out_ds.GetRasterBand(i+1).SetNoDataValue(-9999)
    out_ds.FlushCache()
    del in_ds,out_ds
    return True

In [5]:
if __name__ == '__main__':
    T1=time.time()
    InputPath=r"C:\\2012_mask"
    OutputPathName="G:\LaiParamTwoCycle_2012.tif"
    MosaicImage(InputPath,OutputPathName)
    T2 = time.time()
    print('程序运行时间 :',(T2 - T1))